<a href="https://colab.research.google.com/github/kristopherpaul/ML_InductionAssignment_IITGN/blob/main/1_Paper-Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1 Paper Review**

**Forward Euler's method**\
This algorithm starts from a given initial state $x_{0}$ at $t=0$ and goes in the direction of the gradient at that point $f(x=x_{0},t=0)$ and keeps doing it till $t=T$ using a step size of $Δt$. This can be trivially obtained from the definition of the derivative.
$$x_{t+1}=x_{t}+Δt \,\cdot\,f(x_{t},t)$$

**Motivation for Neural ODE**

If you look at the core of ResNet, a popular deep neural network, its most unique component is the residual block that computes increments on top of the previous layer's activation instead of computing on the activations directly.

If the activation of layer $t$ is $h_t$ then $h_{t+1} = h_{t} + f(h_{t},θ_{t})$. These iterative updates resemble the forward euler's method on an ODE with dynamics $f$. Having $T$ layers corresponds to executing $T$ steps of the foward euler's method with $Δt=1$.\
Therefore we have $$\frac{dh(t)}{dt}=f(h(t),t,θ)$$

Starting from the input layer $h(0)$, the output layer $h(T)$ will be the solution to this ODE initial value problem at some time T. This can be computed by a black-box differential equation solver.\
$$h(T) = ODESolve(start\_state=h(0),dynamics=f,t\_start=0,t\_end=T,θ)$$

**Backward pass using Adjoint method**

Consider optimizing a scalar-valued loss function $L()$, whose input is the result of an ODE solver. If we can compute $\frac{dL}{dθ}$, gradient descent can be performed to update the weights.\
The output of the ODE solver is $\frac{dL}{dh_{T}}$. In order to get $\frac{dL}{dθ}$, we define an adjoint $a(t) = \frac{dL}{dh_{t}}$. Its dynamics are given by another ODE:
$$\frac{da(t)}{dh_{t}}=-a(t)^{T}\frac{df(h(t),t,θ)}{dh(t)}$$
We now have the dynamics that $a_{t}$ follows and an initial value $a_{T}$ (value at the extreme end $t = T$). Which means that we can run an ODE solver backward in time from $t = T → 0$ and calculate all $a_{t}$ in succession.\
Computing the gradients with respect to the parameters $θ$ requires evaluating an integral, which depends on both $h(t)$ and $a(t)$:
$$\frac{dL}{dθ}=- \int_{T}^{T-1} a(t)^{T} \frac{df(h(t),t,θ)}{dθ} \,dt $$
